# Shapes nb

In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import numpy.testing as npt
from operator import itemgetter, attrgetter
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

2017-12-10 10:17:30


In [2]:
from math import sqrt, pi

In [3]:
from pynext.system_of_units import *

In [4]:
from pytest import approx

In [5]:
from collections import namedtuple

## Shapes

### Interface

class Shape(ABC):
    
    """An abstract class representing a geometrical shape.
    In the most general case, the shape is assumed to have a thickness
    (e.g., a sphericall shell or a cylinder shell).

    The interface is defined by 6 methods. To describe them imagine that we want to describe
    a pressure vessel (PV) with gas inside. We can use three shapes.
    A CylinderShell(Rin, Rout, L, ts) to describe the
    PV body; A Disk(R, td) to describe the two end cups. And a Cylinder (Rin, L) to describe
    the volume of gas enclosed by the PV.

    CylinderShell (describes the PV)
    1. inner_volume(): volume enclosed by the CylinderShell
                 = 2 * pi * Rin * L.
    2. shell_volume(): Volume of the shell itself
                 =  2 * pi * (Rout - Rin) * L
    3. inner_surface(): surface in contact with the gas,
                 =  2 * pi * Rin * L
    4. outer_surface(): surface in contact with the air outside the vessel,
                 = 2 * pi * Rout * L
    5. thickness_surface(): surface in contact with the end-cups
                 = pi * (Rout**2 - Rin**2)
    6. thickness() would correspond to the thickness of steel
                 = ts

    Cylinder (describes the volume of gas)
    1. inner_volume()      = 2 * pi * Rin * L
    2. shell_volume()      = 0
    3. inner_surface()     = 2 * pi * Rin * L
    4. outer_surface()     = 2 * pi * Rin * L
    5. thickness_surface() = 0
    6. thickness()         = 0

    Disk (describes the end-cups)
    1. inner_volume()      = pi * R**2 * td
    2. shell_volume()      = pi * R**2 * td
    3. inner_surface()     = pi * R**2
    4. outer_surface()     = pi * R**2
    5. thickness_surface() = 0
    6. thickness()         = 2 * pi * R * td

    Once the interface is defined, the methods volume() and surface()
    as well as the properties V and S come for free.
    """

    @property
    def V(self):
        return self.volume()

    @property
    def S(self):
        return self.surface()

    def volume(self):
        return inner_volume()

    def surface(self):
        return self.inner_surface() + self.outer_surface() + self.thickness_surface()

    @abstractmethod
    def inner_volume(self):
        pass

    @abstractmethod
    def shell_volume(self):
        pass

    @abstractmethod
    def inner_surface(self):
        pass

    @abstractmethod
    def outer_surface(self):
        pass

    @abstractmethod
    def thickness_surface(self):
        pass

    @abstractmethod
    def thickness(self):
        pass


In [6]:
ShapeParams = namedtuple('ShapeParams', 
                         """inner_volume shell_volume 
                            inner_surface outer_surface thickness_surface thickness""")

In [7]:
def test_shape(sp, SP):
    assert sp.inner_volume()      == SP.inner_volume
    assert sp.shell_volume()      == SP.shell_volume
    assert sp.inner_surface()     == SP.inner_surface
    assert sp.outer_surface()     == SP.outer_surface
    assert sp.thickness_surface() == SP.thickness_surface
    assert sp.thickness()         == SP.thickness
    print('all OK')
    

### Sphere

In [8]:
from pynext.Shapes import Sphere

In [9]:
R=1
sphere = ShapeParams(
inner_volume      = (4/3) * pi * R**3, 
shell_volume      = 0,
inner_surface     = 4 * pi * R**2,
outer_surface     = 4 * pi * R**2,
thickness_surface = 0,
thickness         = 0
)
sp = Sphere(R = 1)
sp


        Sphere(R = 1.00e+00)
        

        inner_volume      = 4.19e+00
        shell_volume      = 0.00e+00
        inner_surface     = 1.26e+01
        outer_surface     = 1.26e+01
        thickness_surface = 0.00e+00
        thickness         = 0.00e+00
        volume            = 4.19e+00
        surface           = 1.26e+01
        

In [10]:
test_shape(sp, sphere)

all OK


### SphereShell

In [11]:
from pynext.Shapes import SphereShell

In [12]:
Rin=1
Rout = 2
sphereShell = ShapeParams(
inner_volume      = (4/3) * pi * Rin**3, 
shell_volume      = (4/3) * pi * (Rout**3 - Rin**3),
inner_surface     = 4 * pi * Rin**2,
outer_surface     = 4 * pi * Rout**2,
thickness_surface = 0,
thickness         = Rout - Rin
)
sp = SphereShell(Rin, Rout)
sp


        SphereShell(Rin = 1.00e+00, Rout= 2.00e+00)
        

        inner_volume      = 2.93e+01
        shell_volume      = 2.93e+01
        inner_surface     = 1.26e+01
        outer_surface     = 5.03e+01
        thickness_surface = 0.00e+00
        thickness         = 1.00e+00
        volume            = 2.93e+01
        surface           = 5.03e+01
        

In [13]:
test_shape(sp, sphereShell)

all OK


### Cylinder

In [14]:
from pynext.Shapes import Cylinder

In [15]:
R=1
L = 1
cyl = ShapeParams(
inner_volume      = pi * R**2 * L, 
shell_volume      = 0,
inner_surface     = 2 * pi * R * L,
outer_surface     = 2 * pi * R * L,
thickness_surface = 0,
thickness         = 0
)
sp = Cylinder(R, L)
sp


        Cylinder(R = 1.00e+00, L = 1.00e+00)
        

        inner_volume      = 3.14e+00
        shell_volume      = 0.00e+00
        inner_surface     = 6.28e+00
        outer_surface     = 6.28e+00
        thickness_surface = 0.00e+00
        thickness         = 0.00e+00
        volume            = 3.14e+00
        surface           = 6.28e+00
        

In [16]:
test_shape(sp, cyl)

all OK


### CylinderShell

In [17]:
from pynext.Shapes import CylinderShell

In [18]:
Rin  =1
Rout = 2
L    = 1
scyl = ShapeParams(
inner_volume      = pi * Rin**2 * L, 
shell_volume      = pi * (Rout**2 - Rin**2) * L,
inner_surface     = 2 * pi * Rin * L,
outer_surface     = 2 * pi * Rout * L,
thickness_surface = pi * (Rout**2 - Rin**2),
thickness         = Rout - Rin
)
sp = CylinderShell(Rin, Rout, L)
sp


        CylinderShell(Rin = 1.00e+00, Rout= 2.00e+00, L = 1.00e+00)
        

        inner_volume      = 9.42e+00
        shell_volume      = 9.42e+00
        inner_surface     = 6.28e+00
        outer_surface     = 1.26e+01
        thickness_surface = 9.42e+00
        thickness         = 1.00e+00
        volume            = 9.42e+00
        surface           = 1.26e+01
        

In [19]:
test_shape(sp, scyl)

all OK


### Disk

In [20]:
from pynext.Shapes import Disk

In [21]:
R = 1
t = 1
disk = ShapeParams(
inner_volume      = pi * R**2 * t, 
shell_volume      = 0,
inner_surface     = pi * R**2,
outer_surface     = pi * R**2,
thickness_surface = 2 * pi *  R * t,
thickness         = t
)
sp = Disk(R, t)
sp


        Disk(R = 1.00e+00, t = 1.00e+00)
        

        inner_volume      = 3.14e+00
        shell_volume      = 0.00e+00
        inner_surface     = 3.14e+00
        outer_surface     = 3.14e+00
        thickness_surface = 6.28e+00
        thickness         = 1.00e+00
        volume            = 3.14e+00
        surface           = 3.14e+00
        

In [22]:
test_shape(sp,disk)

all OK


### Brick

In [23]:
from pynext.Shapes import Brick

In [24]:
width  = 1 
heigth = 1 
length =1
brick = ShapeParams(
inner_volume      = width * heigth * length, 
shell_volume      = 0,
inner_surface     = 2 * (width * heigth + width * length + length * heigth),
outer_surface     = 2 * (width * heigth + width * length + length * heigth),
thickness_surface = 0,
thickness         = 0
)
sp = Brick(width, heigth, length)
sp


        Brick(width= 1.00e+00, heigth= 1.00e+00, lengthR= 1.00e+00)
        

        inner_volume      = 1.00e+00
        shell_volume      = 0.00e+00
        inner_surface     = 6.00e+00
        outer_surface     = 6.00e+00
        thickness_surface = 0.00e+00
        thickness         = 0.00e+00
        volume            = 1.00e+00
        surface           = 6.00e+00
        

In [25]:
test_shape(sp,brick)

all OK


### Example: NEXT-100 PV

In [26]:
from pynext.Shapes import FlatPlate  # synonimous for DISK

In [27]:
def Next100PV():
    L    = 130 * cm
    R    =  52 * cm
    ts   =   1 * cm
    tc   =  10 * cm
    Rin  = R
    Rout = Rin + ts
    pv = CylinderShell(Rin, Rout, L)
    fp = FlatPlate(R, tc)
    return pv, fp

In [28]:
pv, fp = Next100PV()

In [29]:
pv


        CylinderShell(Rin = 5.20e+02, Rout= 5.30e+02, L = 1.30e+03)
        

        inner_volume      = 4.29e+07
        shell_volume      = 4.29e+07
        inner_surface     = 4.25e+06
        outer_surface     = 4.33e+06
        thickness_surface = 3.30e+04
        thickness         = 1.00e+01
        volume            = 4.29e+07
        surface           = 4.33e+06
        

In [30]:
fp


        Disk(R = 5.20e+02, t = 1.00e+02)
        

        inner_volume      = 8.49e+07
        shell_volume      = 0.00e+00
        inner_surface     = 8.49e+05
        outer_surface     = 8.49e+05
        thickness_surface = 3.27e+05
        thickness         = 1.00e+02
        volume            = 8.49e+07
        surface           = 8.49e+05
        